In [1]:
%cd ..
%env TOKENIZERS_PARALLELISM=false

d:\Repositories\NLP\AI-CUP-2022-Fall-NLP
env: TOKENIZERS_PARALLELISM=false


In [2]:
import torch
from src import DataLoader, PredictionDataset, SpanPredictionModel
from src.scoring import compute_score

In [3]:
device = torch.device('cuda')
model_path = 'weights/v6-e0+.pt'
dataset_path = 'data/splitted/test.csv'
batch_size = 1
top_k = 3
max_tokens = 10000

In [4]:
model = SpanPredictionModel()
model.load_state_dict(torch.load(model_path))
model = model.requires_grad_(False).to(device)

In [5]:
from tqdm.auto import tqdm

dataset = PredictionDataset(dataset_path, model.tokenizer)
dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=4, pin_memory=True)

answers = []
for xs, encodings, state in tqdm(dataloader):
    encodings.__setstate__(state)
    encodings = encodings.to(device)

    special_tokens_mask = encodings.pop('special_tokens_mask')
    valid_mask: torch.Tensor = encodings.attention_mask.bool()
    valid_mask &= ~special_tokens_mask.bool()
    valid_mask[:, 0] = True

    preds = model(encodings)
    ans = model.decode_answers(xs, encodings, valid_mask, preds, top_k, max_tokens)
    answers.extend(ans)

  0%|          | 0/798 [00:10<?, ?it/s]

In [6]:
compute_score(answers)

  0%|          | 0/798 [00:00<?, ?it/s]

0.8757034201272084